In [1]:
import tensorflow as tf
import numpy as np
import cv2
from matplotlib import pyplot as plt
import pandas as pd
import os
import time

In [2]:
cdnet_root_path = '/home/Datasets/CDnet2014/dataset'
AFH_SS_raw = '/home/Datasets/AFH_SS_results/AFH_SS_raw'
AFH_SS_bin = '/home/Datasets/AFH_SS_results/AFH_SS_bin'
AFH_SS_post_bin = '/home/Datasets/AFH_SS_results/AFH_SS_post_root'
Probability_only_raw = '/home/Datasets/AFH_SS_results/Probability_only_raw'
Probability_only_bin = '/home/Datasets/AFH_SS_results/Probability_only_bin'
SS_only_raw = '/home/Datasets/AFH_SS_results/SS_only_raw'
SS_only_bin = '/home/Datasets/AFH_SS_results/SS_only_bin'

Calculate scores

In [3]:
def calculate_base_scores(uint8_result,uint8_truth):
    pos_mask,neg_mask = calculate_double_mask(uint8_truth)
    
    result = np.float32(uint8_result)/255
    
    TP = np.sum(pos_mask*result)
    TN = np.sum(neg_mask*(1.0-result))
    FP = np.sum(neg_mask*result)
    FN = np.sum(pos_mask*(1.0-result))
    
    return [TP,TN,FP,FN]

In [4]:
def calculate_synthesis_scores(base_scores):
    
    TP = base_scores[0]
    TN = base_scores[1]
    FP = base_scores[2]
    FN = base_scores[3]

    Recall = np.maximum(1e-3,TP) / np.maximum(1e-3,TP + FN)
    Specificity = np.maximum(1e-3,TN) / np.maximum(1e-3,TN + FP)
    PWC = 100.0 * np.maximum(1e-3,FN + FP) / np.maximum(1e-3,TP + FN + FP + TN)
    Precision = np.maximum(1e-3,TP) / np.maximum(1e-3,TP + FP)
    F_Measure = (2 * Precision * Recall) / (Precision + Recall)
    
    return [Recall,Specificity,PWC,Precision,F_Measure]

In [5]:
def calculate_double_mask(ground_truth):
    """Calculate the weight for calculating the loss of the model"""
   
    positive_mask = np.zeros_like(np.float32(ground_truth))
    negative_mask = np.zeros_like(np.float32(ground_truth))

    index_object = np.where(ground_truth==255)
    index_background = np.where(ground_truth==0)
    
    positive_mask[index_object] = 1.0
    negative_mask[index_background] = 1.0
    
    return positive_mask, negative_mask

In [6]:
AFH_SS_raw_score_dict = {}
AFH_SS_bin_score_dict = {}
AFH_SS_post_bin_score_dict = {}
Probability_only_raw_score_dict = {}
Probability_only_bin_score_dict = {}
SS_only_raw_score_dict = {}
SS_only_bin_score_dict = {}
for root, _, files in os.walk(cdnet_root_path):
    
    relative_path_hierarch_list = root.split('/')[len(cdnet_root_path.split('/')):]
    level_above_dataset_root_path = len(relative_path_hierarch_list)
    
    if len(relative_path_hierarch_list) == 2:
        
        print(relative_path_hierarch_list)
        temporalROI = np.loadtxt(root+'/temporalROI.txt').astype(np.int)
        print('{0:06d}-{1:06d}'.format(temporalROI[0],temporalROI[1]))

        AFH_SS_raw_base_scores = np.array([0.0,0.0,0.0,0.0])
        AFH_SS_bin_base_scores = np.array([0.0,0.0,0.0,0.0])
        AFH_SS_post_bin_base_scores = np.array([0.0,0.0,0.0,0.0])
        Probability_only_raw_base_scores = np.array([0.0,0.0,0.0,0.0])
        Probability_only_bin_base_scores = np.array([0.0,0.0,0.0,0.0])
        SS_only_raw_base_scores = np.array([0.0,0.0,0.0,0.0])
        SS_only_bin_base_scores = np.array([0.0,0.0,0.0,0.0])
        
        for file_index in range(temporalROI[0],temporalROI[1]):
            file_index_string = '{0:06d}'.format(file_index)
            
            truth_file_path = cdnet_root_path+'/'+relative_path_hierarch_list[0]+'/'+relative_path_hierarch_list[1]+'/groundtruth/gt'+file_index_string+'.png'
            AFH_SS_raw_file_path = AFH_SS_raw+'/'+relative_path_hierarch_list[0]+'/'+relative_path_hierarch_list[1]+'/raw'+file_index_string+'.png'
            AFH_SS_bin_file_path = AFH_SS_bin+'/'+relative_path_hierarch_list[0]+'/'+relative_path_hierarch_list[1]+'/bin'+file_index_string+'.png'
            AFH_SS_post_bin_file_path = AFH_SS_post_bin+'/'+relative_path_hierarch_list[0]+'/'+relative_path_hierarch_list[1]+'/bin'+file_index_string+'.png'
            Probability_only_raw_path = Probability_only_raw+'/'+relative_path_hierarch_list[0]+'/'+relative_path_hierarch_list[1]+'/raw'+file_index_string+'.png'
            Probability_only_bin_file_path = Probability_only_bin+'/'+relative_path_hierarch_list[0]+'/'+relative_path_hierarch_list[1]+'/bin'+file_index_string+'.png'
            SS_only_raw_file_path = SS_only_raw+'/'+relative_path_hierarch_list[0]+'/'+relative_path_hierarch_list[1]+'/raw'+file_index_string+'.png'
            SS_only_bin_file_path = SS_only_bin+'/'+relative_path_hierarch_list[0]+'/'+relative_path_hierarch_list[1]+'/bin'+file_index_string+'.png'
            
            #print(AFH_SS_post_bin_file_path)
            
            truth = cv2.imread(truth_file_path,0)
            FS_r = cv2.imread(AFH_SS_raw_file_path,0)
            FS_b = cv2.imread(AFH_SS_bin_file_path,0)
            FS_pb = cv2.imread(AFH_SS_post_bin_file_path,0)
            P_r = cv2.imread(Probability_only_raw_path,0)
            P_b = cv2.imread(Probability_only_bin_file_path,0)
            S_r = cv2.imread(SS_only_raw_file_path,0)
            S_b = cv2.imread(SS_only_bin_file_path,0)
            
            AFH_SS_raw_base_scores += np.array(calculate_base_scores(FS_r,truth))
            AFH_SS_bin_base_scores += np.array(calculate_base_scores(FS_b,truth))
            AFH_SS_post_bin_base_scores += np.array(calculate_base_scores(FS_pb,truth))
            Probability_only_raw_base_scores += np.array(calculate_base_scores(P_r,truth))
            Probability_only_bin_base_scores += np.array(calculate_base_scores(P_b,truth))
            SS_only_raw_base_scores += np.array(calculate_base_scores(S_r,truth))
            SS_only_bin_base_scores += np.array(calculate_base_scores(S_b,truth))
        
        AFH_SS_raw_synthesis_scores = calculate_synthesis_scores(AFH_SS_raw_base_scores)
        AFH_SS_bin_synthesis_scores = calculate_synthesis_scores(AFH_SS_bin_base_scores)
        AFH_SS_post_bin_synthesis_scores = calculate_synthesis_scores(AFH_SS_post_bin_base_scores)
        Probability_only_raw_synthesis_scores = calculate_synthesis_scores(Probability_only_raw_base_scores)
        Probability_only_bin_synthesis_scores = calculate_synthesis_scores(Probability_only_bin_base_scores)
        SS_only_raw_synthesis_scores = calculate_synthesis_scores(SS_only_raw_base_scores)
        SS_only_bin_synthesis_scores = calculate_synthesis_scores(SS_only_bin_base_scores)
        
        print(AFH_SS_raw_base_scores)
        print(AFH_SS_bin_base_scores)
        print(AFH_SS_post_bin_base_scores)
        print(Probability_only_raw_base_scores)
        print(Probability_only_bin_base_scores)
        print(SS_only_raw_base_scores)
        print(SS_only_bin_base_scores)
        print(AFH_SS_raw_synthesis_scores)
        print(AFH_SS_bin_synthesis_scores)
        print(AFH_SS_post_bin_synthesis_scores)
        print(Probability_only_raw_synthesis_scores)
        print(Probability_only_bin_synthesis_scores)
        print(SS_only_raw_synthesis_scores)
        print(SS_only_bin_synthesis_scores)
        
        AFH_SS_raw_score_dict[relative_path_hierarch_list[0]+'/'+relative_path_hierarch_list[1]] = AFH_SS_raw_synthesis_scores
        AFH_SS_bin_score_dict[relative_path_hierarch_list[0]+'/'+relative_path_hierarch_list[1]] = AFH_SS_bin_synthesis_scores
        AFH_SS_post_bin_score_dict[relative_path_hierarch_list[0]+'/'+relative_path_hierarch_list[1]] = AFH_SS_post_bin_synthesis_scores
        Probability_only_raw_score_dict[relative_path_hierarch_list[0]+'/'+relative_path_hierarch_list[1]] = Probability_only_raw_synthesis_scores
        Probability_only_bin_score_dict[relative_path_hierarch_list[0]+'/'+relative_path_hierarch_list[1]] = Probability_only_bin_synthesis_scores
        SS_only_raw_score_dict[relative_path_hierarch_list[0]+'/'+relative_path_hierarch_list[1]] = SS_only_raw_synthesis_scores
        SS_only_bin_score_dict[relative_path_hierarch_list[0]+'/'+relative_path_hierarch_list[1]] = SS_only_bin_synthesis_scores

['dynamicBackground', 'overpass']
001000-003000
[1.85450113e+06 1.45477935e+08 7.82939085e+04 1.23390901e+05]
[1.92982400e+06 1.45467392e+08 8.88360000e+04 4.80680000e+04]
[1.92978000e+06 1.45465829e+08 9.03990000e+04 4.81120000e+04]
[7.53540655e+05 1.33282290e+08 1.22739546e+07 1.22435127e+06]
[1.62277800e+06 1.45493249e+08 6.29790000e+04 3.55114000e+05]
[1.83872237e+06 1.45321094e+08 2.35134197e+05 1.39169618e+05]
[1.92961200e+06 1.45467453e+08 8.87750000e+04 4.82800000e+04]
[0.9376149461567805, 0.9994621054088485, 0.13670383995839575, 0.9594918721313521, 0.9484272697286461]
[0.9756973586019864, 0.999389679155467, 0.0927948057032502, 0.9559925891432931, 0.9657444717346352]
[0.9756751126957387, 0.9993789410371365, 0.09388404526356343, 0.9552519850963702, 0.9653555427104722]
[0.38098171390606594, 0.915675520533718, 9.149276325599544, 0.057842328162721446, 0.10043601629418634]
[0.8204583465629064, 0.9995673218462352, 0.28338732762292546, 0.9626405229223429, 0.8858807161930632]
[0.929637

[3.94586758e+06 1.14770259e+08 3.29368202e+05 1.41535487e+05]
[4.04885900e+06 1.14704132e+08 3.95496000e+05 3.85440000e+04]
[4.04845400e+06 1.14687867e+08 4.11761000e+05 3.89490000e+04]
[1.62964452e+06 1.05470154e+08 9.62948738e+06 2.45775834e+06]
[3.65895100e+06 1.14954898e+08 1.44730000e+05 4.28452000e+05]
[3.92866733e+06 1.14708797e+08 3.90831823e+05 1.58735593e+05]
[4.0480650e+06 1.1470463e+08 3.9499800e+05 3.9338000e+04]
[0.9653727599617972, 0.9971384077446731, 0.3950964191745871, 0.9229590555633855, 0.943689581963808]
[0.9905700514483157, 0.9965638811621529, 0.36416713828537267, 0.9110116091086333, 0.949126545783413]
[0.9904709665281354, 0.9964225688027419, 0.37815355934153605, 0.9076813561678081, 0.947270689916185]
[0.3986992659544156, 0.9163378162861151, 10.141409036592425, 0.14473980184873483, 0.212379477922929]
[0.8951774513058781, 0.9987425676128162, 0.4809097056876935, 0.961950016313145, 0.9273633381674812]
[0.9611646818544735, 0.9966044041565963, 0.4610966525560796, 0.9095

[6.24728299e+05 6.73477136e+07 1.22455895e+05 4.61337041e+04]
[6.5341300e+05 6.7404132e+07 6.6037000e+04 1.7449000e+04]
[6.5472400e+05 6.7400766e+07 6.9403000e+04 1.6138000e+04]
[  277153.96388748 62076406.875       5393771.97460938   393708.01729178]
[6.3621600e+05 6.7434085e+07 3.6084000e+04 3.4646000e+04]
[6.06366587e+05 6.57089543e+07 1.76121558e+06 6.44954093e+04]
[6.5354700e+05 6.4336622e+07 3.1335470e+06 1.7315000e+04]
[0.9312321999504596, 0.9981850365014082, 0.24741274853080386, 0.8361101639750421, 0.8811113492125159]
[0.9739901797985279, 0.9990212415208268, 0.12251942592415427, 0.908211828480089, 0.9399516079843949]
[0.9759443820040485, 0.9989713528063047, 0.12553523001435068, 0.9041563151215187, 0.9386798032099178]
[0.4131311233352215, 0.9200569486167799, 8.493383607498643, 0.0488727884816894, 0.08740562358686921]
[0.9483559957189407, 0.9994651858660677, 0.10379942739639499, 0.9463275323516287, 0.9473406781907171]
[0.9038618827715516, 0.9738963814512551, 2.6793122102085833, 0

[ 403642.30730902 6585915.00354004    8882.03557945 1042082.70267868]
[ 481741. 6585457.    9340.  963984.]
[ 502779. 6585238.    9559.  942946.]
[ 278151.12850952 6056595.85375977  538201.23765564 1167573.86752319]
[4.208470e+05 6.591178e+06 3.619000e+03 1.024878e+06]
[ 329617.06527519 6429712.83319092  165084.23394179 1116107.93296814]
[ 471564. 6553957.   40840.  974161.]
[0.2791971533455415, 0.9986531752945897, 13.070852014824464, 0.9784690631411936, 0.43443287811107634]
[0.3332175897905895, 0.9985837319935701, 12.105233963665544, 0.9809807343391416, 0.4974592189408749]
[0.3477694582302997, 0.9985505239964172, 11.846307988461446, 0.9813423950595115, 0.5135473169147264]
[0.1923956003201229, 0.9183900231962818, 21.21473066831917, 0.3407243489917772, 0.24592538978354367]
[0.2910975462138373, 0.999451234056181, 12.791420756015592, 0.9914739932055806, 0.4500577748475958]
[0.2279943043633489, 0.974967503584827, 15.934191293709404, 0.6662951275788256, 0.33973675342021903]
[0.3261782150823

[1.14501947e+07 3.59471455e+08 4.39323765e+05 8.89780442e+05]
[1.21336880e+07 3.59333305e+08 5.77473000e+05 2.06287000e+05]
[1.21329280e+07 3.59331693e+08 5.79085000e+05 2.07047000e+05]
[4.34634182e+06 3.31606052e+08 2.83047975e+07 7.99363269e+06]
[9.12452600e+06 3.59889879e+08 2.08990000e+04 3.21544900e+06]
[1.14079299e+07 3.59359844e+08 5.50936094e+05 9.32045000e+05]
[1.21320350e+07 3.59333338e+08 5.77440000e+05 2.07940000e+05]
[0.9278944702960725, 0.9987793536856994, 0.35704540394090933, 0.9630494903717025, 0.9451451923744808]
[0.9832830293416316, 0.9983955106784826, 0.21054624972108518, 0.9545696101245197, 0.9687135944653368]
[0.9832214408862254, 0.9983910317906622, 0.2111834546107688, 0.9544458458310261, 0.9686199753887795]
[0.35221643405156244, 0.921356086966313, 9.751067802531667, 0.133114552977735, 0.19320890041317454]
[0.7394282403327397, 0.9999419328309196, 0.869399987486392, 0.9977148136909985, 0.8493698976979717]
[0.9244694573920177, 0.9984692425862164, 0.39838229348478366,

[6.08382436e+05 7.05034480e+07 2.34252208e+04 4.80491573e+05]
[6.7346700e+05 7.0518898e+07 7.9750000e+03 4.1540700e+05]
[8.1053700e+05 7.0519795e+07 7.0780000e+03 2.7833700e+05]
[  301610.60204473 64644792.2890625   5882088.87109375   787263.38350821]
[5.1789800e+05 7.0500474e+07 2.6399000e+04 5.7097600e+05]
[  521215.06690475 70233141.171875     293732.41622925   567658.9019382 ]
[1.0613110e+06 7.0433078e+07 9.3795000e+04 2.7563000e+04]
[0.5587261990419305, 0.9996678539722389, 0.7036396501191639, 0.9629234933429329, 0.7071411851179189]
[0.6184985590619301, 0.9998869225351874, 0.5911856229049737, 0.9882968763299004, 0.760843826751834]
[0.7443808925550615, 0.9998996410914177, 0.39853665144343187, 0.9913431138127359, 0.8502928681990822]
[0.2769931195404321, 0.916597915938798, 9.312688585120009, 0.04877510677142458, 0.082944669794783]
[0.4756271157177047, 0.9996256887782335, 0.8341391733301337, 0.9514989059281973, 0.6342238504112552]
[0.47867345700127345, 0.9958351703218162, 1.20279595976

[ 2520127.15035486 76447504.953125      89772.54797363   431081.84656143]
[2.688426e+06 7.649045e+07 4.682700e+04 2.627830e+05]
[2.6887100e+06 7.6485975e+07 5.1302000e+04 2.6249900e+05]
[  995212.20699847 70365671.546875    6171615.32910156  1955996.73518014]
[1.9697960e+06 7.6526767e+07 1.0510000e+04 9.8141300e+05]
[ 2507993.59168983 75357485.6171875   1179791.75153351   443215.4168396 ]
[ 2686855. 74456995.  2080282.   264354.]
[0.8539304241035225, 0.9988270741930644, 0.6552576574068649, 0.9656030658836707, 0.9063398284265799]
[0.9109575092784008, 0.9993881804809962, 0.389502952666629, 0.9828801942635654, 0.9455531400719815]
[0.9110537410261353, 0.999329712239436, 0.3947754143914629, 0.9812767243355138, 0.9448622712068289]
[0.33722187296701317, 0.9193645923312875, 10.224891011744816, 0.13886370252185973, 0.1967204228083157]
[0.6674539146498943, 0.9998626812918887, 1.247882617867448, 0.9946927394049202, 0.7988603907724097]
[0.8498190349925598, 0.9845853969190749, 2.041814157424454, 0.

[6.99461974e+05 1.10806726e+08 2.56377504e+05 8.95099044e+05]
[7.67888000e+05 1.10862992e+08 2.00111000e+05 8.26673000e+05]
[7.85011000e+05 1.10864898e+08 1.98205000e+05 8.09550000e+05]
[3.79865150e+05 1.00510972e+08 1.05521413e+07 1.21469584e+06]
[5.42752000e+05 1.09959319e+08 1.10378400e+06 1.05180900e+06]
[7.20907762e+05 1.10442411e+08 6.20690402e+05 8.73653220e+05]
[1.48229200e+06 1.10476258e+08 5.86845000e+05 1.12269000e+05]
[0.4386548813576479, 0.9976916050731378, 1.0221022743043358, 0.7317776574657129, 0.548511479078495]
[0.48156702691210934, 0.9981982225005905, 0.9114195728397138, 0.7932735467701929, 0.5993131868131868]
[0.4923054056884622, 0.9982153839155745, 0.8945285781888749, 0.7984115392751949, 0.6090604423889266]
[0.23822553805350433, 0.9049896858833801, 10.44477191416708, 0.034747980878009066, 0.06064951994858834]
[0.34037706929995154, 0.9900616499072604, 1.9134011157909327, 0.329632634816366, 0.33491870190864387]
[0.45210422818533574, 0.9944113716293967, 1.3264464961098

In [11]:
np.save('AFH_SS_raw_score_dict',AFH_SS_raw_score_dict)
np.save('AFH_SS_bin_score_dict',AFH_SS_bin_score_dict)
np.save('AFH_SS_post_bin_score_dict',AFH_SS_post_bin_score_dict)
np.save('Probability_only_raw_score_dict',Probability_only_raw_score_dict)
np.save('Probability_only_bin_score_dict',Probability_only_bin_score_dict)
np.save('SS_only_raw_score_dict',SS_only_raw_score_dict)
np.save('SS_only_bin_score_dict',SS_only_bin_score_dict)